In [1]:

import pandas as pd
import numpy as np

# load Billionaires mapping list

This is a list of id mappings between Forbes and Hurun

In [2]:
# all_ids_v1 = pd.read_csv('../source/billionaires list - eo_final.csv', na_values=['', 'n/a'])
all_ids = pd.read_csv('../source/GM-Billionaires_list-v2 - eo_final.csv', na_values=['', 'n/a'])

In [3]:
all_ids = all_ids.iloc[:, :3]

In [4]:
# all_ids_v1[all_ids_v1['Unique Gapminder ID'].duplicated(keep=False)]

In [5]:
# all_ids_v1.shape

In [6]:
all_ids.shape

(6053, 3)

In [7]:
all_ids.drop_duplicates()

Unique Gapminder ID            Forbes_id            Hurun_id
0      abdulla_al_futtaim   abdulla_al_futtaim  abdulla_al_futtaim
1       abdullah_al_rajhi    abdullah_al_rajhi   abdullah_al_rajhi
2        abdulsamad_rabiu     abdulsamad_rabiu    abdulsamad_rabiu
3               abhay_soi            abhay_soi           abhay_soi
4             abhay_vakil          abhay_vakil         abhay_vakil
...                   ...                  ...                 ...
6048         zhu_chongyun         zhu_chongyun                 NaN
6049           zhu_qingyi           zhu_qingyi                 NaN
6050              zhu_yan              zhu_yan                 NaN
6051        ziyad_manasir        ziyad_manasir                 NaN
6052  ziyavudin_magomedov  ziyavudin_magomedov                 NaN

[6053 rows x 3 columns]

In [8]:
all_ids = all_ids.dropna(axis=0, how='all').drop_duplicates()

In [9]:
all_ids.columns = ['gm_id', 'forbes_id', 'hurun_id']

# Create Datapoints


1. convert indicators to use gm id
2. merge indicators from forbes and hurun


for some of forbes records, we need to split them into multiple records (i.e. forbes has entity as a group of people, but hurun have separated record for each person in list)
so we should first divide forbes mapping into 2 parts.


In [10]:
# how to calculate the indicator required by income mountain:
# income by person, year -> income by person, geo, year -> income_level by person, geo, year
# -> population by income_level, geo, year

In [11]:
fm = all_ids[['forbes_id', 'gm_id']].dropna(how='any')

fm1 = fm[fm['forbes_id'].duplicated(keep=False)]  # these ones need to be split to mulitple people.
fm2 = fm[~fm['forbes_id'].duplicated(keep=False)]

In [12]:
fm1

forbes_id               gm_id
2950  chuchat_petaumpai_daonapa_petampai    daonapa_petampai
3112             lee_yeow_chor_yeow_seng       lee_yeow_chor
3230                               safra        joseph_safra
3277                  udo_harald_tschira         udo_tschira
3278                vinod_anil_rai_gupta         vinod_gupta
3301               willi_isolde_liebherr      willi_liebherr
3424                vinod_anil_rai_gupta      anil_rai_gupta
3532                               safra  chelia_cohen_safra
3602  chuchat_petaumpai_daonapa_petampai   chuchat_petaumpai
3625                               safra       david_j_safra
3698                               safra        esther_safra
3816                  udo_harald_tschira      harald_tschira
3917               willi_isolde_liebherr     isolde_liebherr
3924                               safra       jacob_j_safra
4076                                kwee     kwee_liong_keng
4077                                kwee    kwee_liong_phing
4078                                kwee     kwee_liong_seen
4079                                kwee      kwee_liong_tek
4099             lee_yeow_chor_yeow_seng       lee_yeow_seng
5618                                kwee                kwee

In [13]:
fm1.sort_values(by='gm_id')

forbes_id               gm_id
3424                vinod_anil_rai_gupta      anil_rai_gupta
3532                               safra  chelia_cohen_safra
3602  chuchat_petaumpai_daonapa_petampai   chuchat_petaumpai
2950  chuchat_petaumpai_daonapa_petampai    daonapa_petampai
3625                               safra       david_j_safra
3698                               safra        esther_safra
3816                  udo_harald_tschira      harald_tschira
3917               willi_isolde_liebherr     isolde_liebherr
3924                               safra       jacob_j_safra
3230                               safra        joseph_safra
5618                                kwee                kwee
4076                                kwee     kwee_liong_keng
4077                                kwee    kwee_liong_phing
4078                                kwee     kwee_liong_seen
4079                                kwee      kwee_liong_tek
3112             lee_yeow_chor_yeow_seng       lee_yeow_chor
4099             lee_yeow_chor_yeow_seng       lee_yeow_seng
3277                  udo_harald_tschira         udo_tschira
3278                vinod_anil_rai_gupta         vinod_gupta
3301               willi_isolde_liebherr      willi_liebherr

In [14]:
# 1. convert forbes datapoints to use gm_id
# 1.1 split groups

In [15]:
forbes_dps = pd.read_csv('../intermediate/forbes/ddf--datapoints--worth--by--person--year.csv')

In [16]:
forbes_dps

person  year   worth
0           a_jayson_adair  2021  1000.0
1           a_jayson_adair  2022  1100.0
2      a_jerrold_perenchio  2002  2600.0
3      a_jerrold_perenchio  2003  2300.0
4      a_jerrold_perenchio  2004  2700.0
...                    ...   ...     ...
28720   zygmunt_solorz_zak  2018  2800.0
28721   zygmunt_solorz_zak  2019  2700.0
28722   zygmunt_solorz_zak  2020  2400.0
28723   zygmunt_solorz_zak  2021  3200.0
28724   zygmunt_solorz_zak  2022  2900.0

[28725 rows x 3 columns]

In [17]:
fb1 = forbes_dps[forbes_dps.person.isin(fm1.forbes_id.unique())].copy()
fb2 = forbes_dps[~forbes_dps.person.isin(fm1.forbes_id.unique())].copy()

In [18]:
forbes_dps.shape

(28725, 3)

In [19]:
fb1.shape[0] + fb2.shape[0]

28725

In [20]:
fm1g = fm1.groupby('forbes_id')['gm_id']

In [21]:
fm1g.get_group('kwee')

4076     kwee_liong_keng
4077    kwee_liong_phing
4078     kwee_liong_seen
4079      kwee_liong_tek
5618                kwee
Name: gm_id, dtype: object

In [22]:
fb1g = fb1.groupby('person')

In [23]:
fb1g.get_group('lee_yeow_chor_yeow_seng')

person  year   worth
14795  lee_yeow_chor_yeow_seng  2020  3600.0
14796  lee_yeow_chor_yeow_seng  2021  5100.0

In [24]:
def create_rec(df, names):
    """split the record by names evenly. create dataframe which row number equal to number of names."""
    recs = list()
    ser = df.set_index('year')['worth']
    num = len(names)
    
    for y in ser.index:
        income = int(ser.loc[y] / num)
        for n in names:
            rec = dict(person=n, year=y, worth=income)
            recs.append(rec)
    return pd.DataFrame.from_records(recs)

In [25]:
fm1g.get_group('lee_yeow_chor_yeow_seng').values

array(['lee_yeow_chor', 'lee_yeow_seng'], dtype=object)

In [26]:
fb3dfs = list()

for g, df in fb1g:
    new_df = create_rec(df, fm1g.get_group(g).values)
    fb3dfs.append(new_df)

In [27]:
fb3 = pd.concat(fb3dfs, ignore_index=True)

In [28]:
fb3

person  year  worth
0     daonapa_petampai  2018   1000
1    chuchat_petaumpai  2018   1000
2     daonapa_petampai  2019   1200
3    chuchat_petaumpai  2019   1200
4     daonapa_petampai  2021   1750
..                 ...   ...    ...
109    isolde_liebherr  2016   2500
110     willi_liebherr  2017   2400
111    isolde_liebherr  2017   2400
112     willi_liebherr  2018   3550
113    isolde_liebherr  2018   3550

[114 rows x 3 columns]

In [29]:
fb3[fb3['person'].isin(['lee_yeow_chor', 'lee_yeow_seng'])]

person  year  worth
58  lee_yeow_chor  2020   1800
59  lee_yeow_seng  2020   1800
60  lee_yeow_chor  2021   2550
61  lee_yeow_seng  2021   2550

In [30]:
# 1.2 convert remaining records to use gm id

In [31]:
np.any(fm2.set_index('forbes_id')['gm_id'].index.duplicated())  # Should be false.

False

In [32]:
fb2_map = fm2.set_index('forbes_id')['gm_id'].to_dict()

In [33]:
# NOTE: if below failed, then there are not existing entries, we should exam what happened

fb2['gm_id'] = fb2['person'].map(lambda x: fb2_map[x])

In [34]:
forbes_dps[forbes_dps['person'] == 'xiangdong_qi']  # should be non empty dataframe

person  year   worth
27503  xiangdong_qi  2022  2100.0

In [35]:
def _f(x):
    if x not in fb2_map:
        print(x)
        return None
    return fb2_map[x]
_ = fb2['person'].map(_f)

In [36]:
fb2

person  year   worth                gm_id
0           a_jayson_adair  2021  1000.0       a_jayson_adair
1           a_jayson_adair  2022  1100.0       a_jayson_adair
2      a_jerrold_perenchio  2002  2600.0  a_jerrold_perenchio
3      a_jerrold_perenchio  2003  2300.0  a_jerrold_perenchio
4      a_jerrold_perenchio  2004  2700.0  a_jerrold_perenchio
...                    ...   ...     ...                  ...
28720   zygmunt_solorz_zak  2018  2800.0   zygmunt_solorz_zak
28721   zygmunt_solorz_zak  2019  2700.0   zygmunt_solorz_zak
28722   zygmunt_solorz_zak  2020  2400.0   zygmunt_solorz_zak
28723   zygmunt_solorz_zak  2021  3200.0   zygmunt_solorz_zak
28724   zygmunt_solorz_zak  2022  2900.0   zygmunt_solorz_zak

[28686 rows x 4 columns]

In [37]:
fb2[pd.isnull(fb2.gm_id)]['person'].unique()

array([], dtype=object)

In [38]:
fb4 = fb2[['gm_id', 'year', 'worth']].copy()
# uncomment to drop all unknown entities
# fb4 = fb4.dropna(how='any')
fb4.columns = ['person', 'year', 'worth']

In [39]:
fb_final = pd.concat([fb4, fb3], ignore_index=True).sort_values(by=['person', 'year'])

In [40]:
fb_final

person  year   worth
0           a_jayson_adair  2021  1000.0
1           a_jayson_adair  2022  1100.0
2      a_jerrold_perenchio  2002  2600.0
3      a_jerrold_perenchio  2003  2300.0
4      a_jerrold_perenchio  2004  2700.0
...                    ...   ...     ...
28681   zygmunt_solorz_zak  2018  2800.0
28682   zygmunt_solorz_zak  2019  2700.0
28683   zygmunt_solorz_zak  2020  2400.0
28684   zygmunt_solorz_zak  2021  3200.0
28685   zygmunt_solorz_zak  2022  2900.0

[28800 rows x 3 columns]

In [41]:
fb_final[fb_final['person'] == 'adam_neumann']

person  year   worth
153  adam_neumann  2016  1500.0
154  adam_neumann  2017  2500.0
155  adam_neumann  2018  2500.0
156  adam_neumann  2019  4100.0
157  adam_neumann  2022  1400.0

In [42]:
# done for forbes datapoints

In [43]:
# now convert hurun data to use gm_id

In [44]:
hurun_dps = pd.read_csv('../intermediate/hurun/ddf--datapoints--wealth--by--person--year.csv')

In [45]:
hurun_dps

person  year  wealth
0      abdulla_ahmed_al_ghurair  2017     1.3
1      abdulla_ahmed_al_ghurair  2018     1.3
2      abdulla_ahmed_al_ghurair  2019     1.2
3      abdulla_ahmed_al_ghurair  2020     1.2
4      abdulla_ahmed_al_ghurair  2021     1.2
...                         ...   ...     ...
17977        zygmunt_solorz_zak  2018     3.0
17978        zygmunt_solorz_zak  2019     2.6
17979        zygmunt_solorz_zak  2020     2.8
17980        zygmunt_solorz_zak  2021     3.2
17981        zygmunt_solorz_zak  2022     3.2

[17982 rows x 3 columns]

In [46]:
hr = hurun_dps.copy()

In [47]:
hr_map = all_ids[['hurun_id', 'gm_id']].dropna(how='any').set_index('hurun_id')['gm_id'].to_dict()

In [48]:
# if this failed, we should check which ones are missing.

hr['gm_id'] = hr['person'].map(lambda x: hr_map[x])

In [49]:
hr[hr['person'] == 'zhuo_jun']

person  year  wealth       gm_id
17920  zhuo_jun  2017     1.0  zhuo_jun_1
17921  zhuo_jun  2018     1.3  zhuo_jun_1
17922  zhuo_jun  2019     1.1  zhuo_jun_1
17923  zhuo_jun  2020     1.7  zhuo_jun_1
17924  zhuo_jun  2021     1.5  zhuo_jun_1
17925  zhuo_jun  2022     1.7  zhuo_jun_1

In [50]:
def _f(x):
    if x not in hr_map:
        print(x)
        return None
    return hr_map[x]
_ = hr['person'].map(_f)

In [51]:
len(hr[pd.isnull(hr.gm_id)]['person'].unique())  # dropped records.

0

In [52]:
len(hr.person.unique())  # total records

5028

In [53]:
hr2 = hr[['gm_id', 'year', 'wealth']].dropna()

In [54]:
hr2

gm_id  year  wealth
0      abdulla_bin_ahmad_al_ghurair  2017     1.3
1      abdulla_bin_ahmad_al_ghurair  2018     1.3
2      abdulla_bin_ahmad_al_ghurair  2019     1.2
3      abdulla_bin_ahmad_al_ghurair  2020     1.2
4      abdulla_bin_ahmad_al_ghurair  2021     1.2
...                             ...   ...     ...
17977            zygmunt_solorz_zak  2018     3.0
17978            zygmunt_solorz_zak  2019     2.6
17979            zygmunt_solorz_zak  2020     2.8
17980            zygmunt_solorz_zak  2021     3.2
17981            zygmunt_solorz_zak  2022     3.2

[17982 rows x 3 columns]

In [55]:
hr2.columns = ['person', 'year', 'wealth']

In [56]:
hr2[hr2['person'] == 'adam_neuman']

person  year  wealth
66  adam_neuman  2017     2.6
67  adam_neuman  2018     3.2

In [57]:
# in hurun list, there are many entities without an country property. let's drop them

In [57]:
hurun_ent = pd.read_csv('../intermediate/hurun/ddf--entities--person.csv')

In [58]:
hurun_ent.columns

Index(['person', 'name', 'birth', 'birth_1', 'birth_2', 'birth_3', 'age',
       'sex', 'permanent_country', 'industry', 'company', 'country', 'geo',
       'birth_comb'],
      dtype='object')

In [59]:
hascountry = ~pd.isnull(hurun_ent['geo'])

ppl_hascountry = hurun_ent[hascountry]['person'].values

In [80]:
hr3 = hr2[hr2['person'].isin(ppl_hascountry)]

In [81]:
hr3

person  year  wealth
6      abdulla_al_futtaim  2017     3.1
7      abdulla_al_futtaim  2018     3.1
8      abdulla_al_futtaim  2019     3.1
9      abdulla_al_futtaim  2020     1.2
10     abdulla_al_futtaim  2021     1.5
...                   ...   ...     ...
17977  zygmunt_solorz_zak  2018     3.0
17978  zygmunt_solorz_zak  2019     2.6
17979  zygmunt_solorz_zak  2020     2.8
17980  zygmunt_solorz_zak  2021     3.2
17981  zygmunt_solorz_zak  2022     3.2

[14234 rows x 3 columns]

In [82]:
hr3 = hr3.copy()

In [83]:
hr3.loc[:, 'wealth'] = hr3.loc[:, 'wealth'] * 1000

In [62]:
# done with hurun data

In [68]:
# now, concat all values, drop duplicated

In [84]:
d1 = fb_final.copy()
d1['source'] = 'forbes'

d2 = hr3.copy()
d2 = d2.rename(columns={'wealth': 'worth'})
d2['source'] = 'hurun'

dps = pd.concat([d1, d2], ignore_index=True)

In [85]:
dps.sort_values(by=['person', 'year', 'source'])

person  year   worth  source
0           a_jayson_adair  2021  1000.0  forbes
1           a_jayson_adair  2022  1100.0  forbes
2      a_jerrold_perenchio  2002  2600.0  forbes
3      a_jerrold_perenchio  2003  2300.0  forbes
4      a_jerrold_perenchio  2004  2700.0  forbes
...                    ...   ...     ...     ...
43031   zygmunt_solorz_zak  2020  2800.0   hurun
28798   zygmunt_solorz_zak  2021  3200.0  forbes
43032   zygmunt_solorz_zak  2021  3200.0   hurun
28799   zygmunt_solorz_zak  2022  2900.0  forbes
43033   zygmunt_solorz_zak  2022  3200.0   hurun

[43034 rows x 4 columns]

In [86]:
dps[dps.duplicated(subset=['person', 'year'], keep=False)]

person  year   worth  source
43     abdulla_al_futtaim  2017  4100.0  forbes
44     abdulla_al_futtaim  2018  3300.0  forbes
45     abdulla_al_futtaim  2019  2500.0  forbes
46     abdulla_al_futtaim  2020  2100.0  forbes
47     abdulla_al_futtaim  2021  2200.0  forbes
...                   ...   ...     ...     ...
43029  zygmunt_solorz_zak  2018  3000.0   hurun
43030  zygmunt_solorz_zak  2019  2600.0   hurun
43031  zygmunt_solorz_zak  2020  2800.0   hurun
43032  zygmunt_solorz_zak  2021  3200.0   hurun
43033  zygmunt_solorz_zak  2022  3200.0   hurun

[18406 rows x 4 columns]

In [87]:
# let's see how different are they
dpsg = dps.groupby(['person'])

In [88]:
dpsg.get_group('mark_zuckerberg').sort_values(by=['year', 'source'])   ## TODO: maybe check more people

person  year     worth  source
16995  mark_zuckerberg  2008    1500.0  forbes
16996  mark_zuckerberg  2010    4000.0  forbes
16997  mark_zuckerberg  2011   13500.0  forbes
16998  mark_zuckerberg  2012   17500.0  forbes
36886  mark_zuckerberg  2012   26000.0   hurun
16999  mark_zuckerberg  2013   13300.0  forbes
36887  mark_zuckerberg  2013   17000.0   hurun
17000  mark_zuckerberg  2014   28500.0  forbes
36888  mark_zuckerberg  2014   31000.0   hurun
17001  mark_zuckerberg  2015   33400.0  forbes
36889  mark_zuckerberg  2015   44000.0   hurun
17002  mark_zuckerberg  2016   44600.0  forbes
36890  mark_zuckerberg  2016   47000.0   hurun
17003  mark_zuckerberg  2017   56000.0  forbes
36891  mark_zuckerberg  2017   58000.0   hurun
17004  mark_zuckerberg  2018   71000.0  forbes
36892  mark_zuckerberg  2018   79000.0   hurun
17005  mark_zuckerberg  2019   62300.0  forbes
36893  mark_zuckerberg  2019   80000.0   hurun
17006  mark_zuckerberg  2020   54700.0  forbes
36894  mark_zuckerberg  2020   84000.0   hurun
17007  mark_zuckerberg  2021   97000.0  forbes
36895  mark_zuckerberg  2021  101000.0   hurun
17008  mark_zuckerberg  2022   67300.0  forbes
36896  mark_zuckerberg  2022   76000.0   hurun

In [69]:
# But I am going to use Forbes data for these duplicated people for now.

In [91]:
def keep_one_source(df):
    if len(df['source'].unique()) == 2:
        df_new = df[df['source'] == 'forbes'][['person', 'year', 'worth']].set_index(['person', 'year'])
    else:
        df_new = df[['person', 'year', 'worth']].set_index(['person', 'year'])
    return df_new

In [92]:
keep_one_source(dpsg.get_group('mark_zuckerberg'))

worth
person          year         
mark_zuckerberg 2008   1500.0
                2010   4000.0
                2011  13500.0
                2012  17500.0
                2013  13300.0
                2014  28500.0
                2015  33400.0
                2016  44600.0
                2017  56000.0
                2018  71000.0
                2019  62300.0
                2020  54700.0
                2021  97000.0
                2022  67300.0

In [93]:
dps2 = dpsg.apply(keep_one_source)

In [94]:
dps2

worth
person              person              year        
a_jayson_adair      a_jayson_adair      2021  1000.0
                                        2022  1100.0
a_jerrold_perenchio a_jerrold_perenchio 2002  2600.0
                                        2003  2300.0
                                        2004  2700.0
...                                              ...
zygmunt_solorz_zak  zygmunt_solorz_zak  2018  2800.0
                                        2019  2700.0
                                        2020  2400.0
                                        2021  3200.0
                                        2022  2900.0

[32436 rows x 1 columns]

In [95]:
dps2 = dps2.reset_index(0, drop=True)

In [96]:
dps2

worth
person              year        
a_jayson_adair      2021  1000.0
                    2022  1100.0
a_jerrold_perenchio 2002  2600.0
                    2003  2300.0
                    2004  2700.0
...                          ...
zygmunt_solorz_zak  2018  2800.0
                    2019  2700.0
                    2020  2400.0
                    2021  3200.0
                    2022  2900.0

[32436 rows x 1 columns]

In [97]:
dps2['worth'] = dps2['worth'].astype(int)

In [98]:
dps2 = dps2.reset_index()

In [99]:
dps2.columns = ['person', 'time', 'worth']

In [100]:
# now we should convert them to 2017 PPP dollars.
rates = [1.652, 1.671, 1.658, 1.620, 1.563, 1.542, 1.513, 1.462, 1.438, 1.437, 1.449, 1.427, 1.376, 1.347, 1.337, 1.279, 1.202, 1.162, 1.142, 1.137, 1.111, 1.102, 1.091, 1.068, 1.040, 1.028, 1.012, 1.019, 1.019, 1.000, 0.955, 0.935, 0.917, 0.88, 0.86]
years = list(range(1988, 2023))
rs = pd.Series(rates, index=years)

In [101]:
rs

1988    1.652
1989    1.671
1990    1.658
1991    1.620
1992    1.563
1993    1.542
1994    1.513
1995    1.462
1996    1.438
1997    1.437
1998    1.449
1999    1.427
2000    1.376
2001    1.347
2002    1.337
2003    1.279
2004    1.202
2005    1.162
2006    1.142
2007    1.137
2008    1.111
2009    1.102
2010    1.091
2011    1.068
2012    1.040
2013    1.028
2014    1.012
2015    1.019
2016    1.019
2017    1.000
2018    0.955
2019    0.935
2020    0.917
2021    0.880
2022    0.860
dtype: float64

In [102]:
rs.index.name = 'time'
rs.name = 'rate'
dfrs = rs.reset_index()

In [103]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [104]:
X = rs.index.values
Y = rs.values
X = sm.add_constant(X)

model = sm.OLS(Y, X)
modelres = model.fit()

In [105]:
b, a = modelres.params

def get_inflation_rate(y):
    if y in rs.index:
        return rs.loc[y]
    else:
        return a * y + b

In [106]:
dps2['rates'] = dps2['time'].map(get_inflation_rate)

In [107]:
dps2['worth_2017'] = dps2['worth'] * dps2['rates']

In [108]:
dps2

person  time  worth  rates  worth_2017
0           a_jayson_adair  2021   1000  0.880       880.0
1           a_jayson_adair  2022   1100  0.860       946.0
2      a_jerrold_perenchio  2002   2600  1.337      3476.2
3      a_jerrold_perenchio  2003   2300  1.279      2941.7
4      a_jerrold_perenchio  2004   2700  1.202      3245.4
...                    ...   ...    ...    ...         ...
32431   zygmunt_solorz_zak  2018   2800  0.955      2674.0
32432   zygmunt_solorz_zak  2019   2700  0.935      2524.5
32433   zygmunt_solorz_zak  2020   2400  0.917      2200.8
32434   zygmunt_solorz_zak  2021   3200  0.880      2816.0
32435   zygmunt_solorz_zak  2022   2900  0.860      2494.0

[32436 rows x 5 columns]

In [122]:
dps3 = dps2.set_index(['person', 'time'])['worth_2017']

In [123]:
dps3.name = 'worth'

In [124]:
dps3

person               time
a_jayson_adair       2021     880.0
                     2022     946.0
a_jerrold_perenchio  2002    3476.2
                     2003    2941.7
                     2004    3245.4
                              ...  
zygmunt_solorz_zak   2018    2674.0
                     2019    2524.5
                     2020    2200.8
                     2021    2816.0
                     2022    2494.0
Name: worth, Length: 32436, dtype: float64

In [137]:
np.round(dps3).astype(int).to_csv('../../ddf--datapoints--worth--by--person--time.csv')

In [80]:
dps2 

person  time  annual_income
0           a_jayson_adair  2021       30000000
1           a_jayson_adair  2022       33000000
2      a_jerrold_perenchio  2002       78000000
3      a_jerrold_perenchio  2003       69000000
4      a_jerrold_perenchio  2004       81000000
...                    ...   ...            ...
32431   zygmunt_solorz_zak  2018       84000000
32432   zygmunt_solorz_zak  2019       81000000
32433   zygmunt_solorz_zak  2020       72000000
32434   zygmunt_solorz_zak  2021       96000000
32435   zygmunt_solorz_zak  2022       87000000

[32436 rows x 3 columns]

In [81]:
# now get the daily / annual income 

In [132]:
dps4 = dps3.copy()

In [127]:
dps4 * 0.03 * 1e6

person               time
a_jayson_adair       2021     26400000.0
                     2022     28380000.0
a_jerrold_perenchio  2002    104286000.0
                     2003     88251000.0
                     2004     97362000.0
                                ...     
zygmunt_solorz_zak   2018     80220000.0
                     2019     75735000.0
                     2020     66024000.0
                     2021     84480000.0
                     2022     74820000.0
Name: worth, Length: 32436, dtype: float64

In [133]:
dps4 = dps4 * 0.03 * 1e6
dps4 = np.round(dps4).astype(int)

dps4.name = 'annual_income'

In [134]:
dps4.to_csv('../../ddf--datapoints--annual_income--by--person--time.csv')

In [135]:
dps5 = dps4.copy()
dps5 = dps5 / 365.
dps5 = dps5.astype(int)
dps5.name = 'daily_income'

In [136]:
dps5.to_csv('../../ddf--datapoints--daily_income--by--person--time.csv')

In [86]:
# 2. combine the entities

In [87]:
# For forbes list, we should also take care of entity spliting.

In [88]:
forbes_ent = pd.read_csv('../intermediate/forbes/ddf--entities--person.csv')

In [89]:
forbes_ent 

person                   name               country  \
0            a_jayson_adair        A. Jayson Adair         United States   
1       a_jerrold_perenchio   A. Jerrold Perenchio         United States   
2     abdul_aziz_al_ghurair  Abdul Aziz Al Ghurair  United Arab Emirates   
3     abdul_majeed_alhokair  Abdul Majeed Alhokair          Saudi Arabia   
4              abdul_rasyid           Abdul Rasyid             Indonesia   
...                     ...                    ...                   ...   
4214            zong_yanmin            Zong Yanmin                 China   
4215               zugen_ni               Zugen Ni                 China   
4216                zuo_hui                Zuo Hui                 China   
4217            zuowen_song            Zuowen Song                 China   
4218     zygmunt_solorz_zak     Zygmunt Solorz-Zak                Poland   

           state headquarters                                        source  \
0          Texas           TX                                  damaged cars   
1     California           CA  television, Univision; television; Univision   
2            NaN          NaN                                       banking   
3            NaN          NaN                           retail, real estate   
4            NaN          NaN                              timber, palm oil   
...          ...          ...                                           ...   
4214         NaN          NaN                       Semiconductor materials   
4215     Jiangsu          NaN                                    appliances   
4216         NaN          NaN                          real estate services   
4217         NaN          NaN                         aluminum, diversified   
4218         NaN          NaN                   TV broadcasting; TV station   

                          industry                      image_uri   age  \
0                       Automotive                         no-pic  52.0   
1     Media & Entertainment; Media    a-jerrold-perenchio; no-pic  91.0   
2                          Finance  abdul-aziz-al-ghurair; no-pic  67.0   
3                 Fashion & Retail                         no-pic  52.0   
4                    Manufacturing                   abdul-rasyid  64.0   
...                            ...                            ...   ...   
4214                 Manufacturing                         no-pic   NaN   
4215                 Manufacturing                         no-pic  65.0   
4216                   Real Estate                         no-pic  51.0   
4217                   Diversified                    zuowen-song  75.0   
4218  Media & Entertainment; Media     zygmunt-solorz-zak; no-pic  65.0   

     gender   last_name  
0         M       Adair  
1         M   Perenchio  
2         M  Al Ghurair  
3         M    Alhokair  
4         M       Abdul  
...     ...         ...  
4214      M        Zong  
4215      M          Ni  
4216      M         Zuo  
4217      M        Song  
4218      M  Solorz-Zak  

[4219 rows x 11 columns]

In [90]:
fbent1 = forbes_ent[forbes_ent.person.isin(fm1.forbes_id.unique())].copy()
fbent2 = forbes_ent[~forbes_ent.person.isin(fm1.forbes_id.unique())].copy()

In [91]:
fbent2

person                   name               country  \
0            a_jayson_adair        A. Jayson Adair         United States   
1       a_jerrold_perenchio   A. Jerrold Perenchio         United States   
2     abdul_aziz_al_ghurair  Abdul Aziz Al Ghurair  United Arab Emirates   
3     abdul_majeed_alhokair  Abdul Majeed Alhokair          Saudi Arabia   
4              abdul_rasyid           Abdul Rasyid             Indonesia   
...                     ...                    ...                   ...   
4214            zong_yanmin            Zong Yanmin                 China   
4215               zugen_ni               Zugen Ni                 China   
4216                zuo_hui                Zuo Hui                 China   
4217            zuowen_song            Zuowen Song                 China   
4218     zygmunt_solorz_zak     Zygmunt Solorz-Zak                Poland   

           state headquarters                                        source  \
0          Texas           TX                                  damaged cars   
1     California           CA  television, Univision; television; Univision   
2            NaN          NaN                                       banking   
3            NaN          NaN                           retail, real estate   
4            NaN          NaN                              timber, palm oil   
...          ...          ...                                           ...   
4214         NaN          NaN                       Semiconductor materials   
4215     Jiangsu          NaN                                    appliances   
4216         NaN          NaN                          real estate services   
4217         NaN          NaN                         aluminum, diversified   
4218         NaN          NaN                   TV broadcasting; TV station   

                          industry                      image_uri   age  \
0                       Automotive                         no-pic  52.0   
1     Media & Entertainment; Media    a-jerrold-perenchio; no-pic  91.0   
2                          Finance  abdul-aziz-al-ghurair; no-pic  67.0   
3                 Fashion & Retail                         no-pic  52.0   
4                    Manufacturing                   abdul-rasyid  64.0   
...                            ...                            ...   ...   
4214                 Manufacturing                         no-pic   NaN   
4215                 Manufacturing                         no-pic  65.0   
4216                   Real Estate                         no-pic  51.0   
4217                   Diversified                    zuowen-song  75.0   
4218  Media & Entertainment; Media     zygmunt-solorz-zak; no-pic  65.0   

     gender   last_name  
0         M       Adair  
1         M   Perenchio  
2         M  Al Ghurair  
3         M    Alhokair  
4         M       Abdul  
...     ...         ...  
4214      M        Zong  
4215      M          Ni  
4216      M         Zuo  
4217      M        Song  
4218      M  Solorz-Zak  

[4212 rows x 11 columns]

In [92]:
fbent2.columns

Index(['person', 'name', 'country', 'state', 'headquarters', 'source',
       'industry', 'image_uri', 'age', 'gender', 'last_name'],
      dtype='object')

In [93]:
cols_rec = ['country', 'state', 'headquarters', 'source', 'industry', 'last_name']

In [94]:
def create_entrec():
    res = []
    
    for _, row in fbent1.iterrows():
        name = row['person']
        newnames = fm1g.get_group(name)
        rec = dict([(c, row[c]) for c in cols_rec])
        for n in newnames:
            rec_ = rec.copy()
            rec_['person'] = n
            res.append(rec_)
            
    return pd.DataFrame.from_records(res)

In [95]:
create_entrec()

country  state  headquarters  \
0      Thailand    NaN           NaN   
1      Thailand    NaN           NaN   
2     Singapore    NaN           NaN   
3     Singapore    NaN           NaN   
4     Singapore    NaN           NaN   
5     Singapore    NaN           NaN   
6     Singapore    NaN           NaN   
7      Malaysia    NaN           NaN   
8      Malaysia    NaN           NaN   
9        Brazil    NaN           NaN   
10       Brazil    NaN           NaN   
11       Brazil    NaN           NaN   
12       Brazil    NaN           NaN   
13       Brazil    NaN           NaN   
14      Germany    NaN           NaN   
15      Germany    NaN           NaN   
16        India    NaN           NaN   
17        India    NaN           NaN   
18  Switzerland    NaN           NaN   
19  Switzerland    NaN           NaN   

                                          source  \
0                               motorcycle loans   
1                               motorcycle loans   
2   Real Estate; Pontiac Land Group; real estate   
3   Real Estate; Pontiac Land Group; real estate   
4   Real Estate; Pontiac Land Group; real estate   
5   Real Estate; Pontiac Land Group; real estate   
6   Real Estate; Pontiac Land Group; real estate   
7                             palm oil, property   
8                             palm oil, property   
9                                        banking   
10                                       banking   
11                                       banking   
12                                       banking   
13                                       banking   
14                            software; Software   
15                            software; Software   
16                          electrical equipment   
17                          electrical equipment   
18                    Construction; construction   
19                    Construction; construction   

                                          industry            last_name  \
0   Finance & Investments; Finance and Investments  Chuchat Petaumpai &   
1   Finance & Investments; Finance and Investments  Chuchat Petaumpai &   
2                                      Real Estate                 Kwee   
3                                      Real Estate                 Kwee   
4                                      Real Estate                 Kwee   
5                                      Real Estate                 Kwee   
6                                      Real Estate                 Kwee   
7                                  Food & Beverage                  Lee   
8                                  Food & Beverage                  Lee   
9                            Finance & Investments                Safra   
10                           Finance & Investments                Safra   
11                           Finance & Investments                Safra   
12                           Finance & Investments                Safra   
13                           Finance & Investments                Safra   
14                                      Technology              Tschira   
15                                      Technology              Tschira   
16                                   Manufacturing                Gupta   
17                                   Manufacturing                Gupta   
18                      Construction & Engineering             Liebherr   
19                      Construction & Engineering             Liebherr   

                person  
0     daonapa_petampai  
1    chuchat_petaumpai  
2      kwee_liong_keng  
3     kwee_liong_phing  
4      kwee_liong_seen  
5       kwee_liong_tek  
6                 kwee  
7        lee_yeow_chor  
8        lee_yeow_seng  
9         joseph_safra  
10  chelia_cohen_safra  
11       david_j_safra  
12        esther_safra  
13       jacob_j_safra  
14         udo_tschira  
15      harald_tschira  
16         vinod_gupta  
17      anil_rai_gupta  
18      willi_liebherr 

In [96]:
fbent3 = create_entrec()

In [97]:
fbent4 = fbent2.copy()

In [98]:
len(fm2)

4217

In [99]:
fm2_mapping = fm2.set_index('forbes_id')['gm_id'].to_dict()

In [100]:
def _f(x):
    if x in fm2_mapping:
        return fm2_mapping[x]
    print(x)
    return None

fbent4['gm_id'] = fbent4['person'].map(_f)

In [101]:
fbent4[pd.isnull(fbent4.gm_id)]

Empty DataFrame
Columns: [person, name, country, state, headquarters, source, industry, image_uri, age, gender, last_name, gm_id]
Index: []

In [102]:
fbent4 = fbent4[~pd.isnull(fbent4.gm_id)]
fbent4 = fbent4.set_index(['gm_id']).drop(columns=['person'])
fbent4.index.name = 'person'
fbent4 = fbent4.reset_index()

In [103]:
fbent4

person                   name               country  \
0            a_jayson_adair        A. Jayson Adair         United States   
1       a_jerrold_perenchio   A. Jerrold Perenchio         United States   
2     abdul_aziz_al_ghurair  Abdul Aziz Al Ghurair  United Arab Emirates   
3     abdul_majeed_alhokair  Abdul Majeed Alhokair          Saudi Arabia   
4              abdul_rasyid           Abdul Rasyid             Indonesia   
...                     ...                    ...                   ...   
4207            zong_yanmin            Zong Yanmin                 China   
4208               zugen_ni               Zugen Ni                 China   
4209                zuo_hui                Zuo Hui                 China   
4210            zuowen_song            Zuowen Song                 China   
4211     zygmunt_solorz_zak     Zygmunt Solorz-Zak                Poland   

           state headquarters                                        source  \
0          Texas           TX                                  damaged cars   
1     California           CA  television, Univision; television; Univision   
2            NaN          NaN                                       banking   
3            NaN          NaN                           retail, real estate   
4            NaN          NaN                              timber, palm oil   
...          ...          ...                                           ...   
4207         NaN          NaN                       Semiconductor materials   
4208     Jiangsu          NaN                                    appliances   
4209         NaN          NaN                          real estate services   
4210         NaN          NaN                         aluminum, diversified   
4211         NaN          NaN                   TV broadcasting; TV station   

                          industry                      image_uri   age  \
0                       Automotive                         no-pic  52.0   
1     Media & Entertainment; Media    a-jerrold-perenchio; no-pic  91.0   
2                          Finance  abdul-aziz-al-ghurair; no-pic  67.0   
3                 Fashion & Retail                         no-pic  52.0   
4                    Manufacturing                   abdul-rasyid  64.0   
...                            ...                            ...   ...   
4207                 Manufacturing                         no-pic   NaN   
4208                 Manufacturing                         no-pic  65.0   
4209                   Real Estate                         no-pic  51.0   
4210                   Diversified                    zuowen-song  75.0   
4211  Media & Entertainment; Media     zygmunt-solorz-zak; no-pic  65.0   

     gender   last_name  
0         M       Adair  
1         M   Perenchio  
2         M  Al Ghurair  
3         M    Alhokair  
4         M       Abdul  
...     ...         ...  
4207      M        Zong  
4208      M          Ni  
4209      M         Zuo  
4210      M        Song  
4211      M  Solorz-Zak  

[4212 rows x 11 columns]

In [104]:
fbent_final = pd.concat([fbent4, fbent3], ignore_index=True)

In [105]:
fbent_final

person                   name               country  \
0            a_jayson_adair        A. Jayson Adair         United States   
1       a_jerrold_perenchio   A. Jerrold Perenchio         United States   
2     abdul_aziz_al_ghurair  Abdul Aziz Al Ghurair  United Arab Emirates   
3     abdul_majeed_alhokair  Abdul Majeed Alhokair          Saudi Arabia   
4              abdul_rasyid           Abdul Rasyid             Indonesia   
...                     ...                    ...                   ...   
4227         harald_tschira                    NaN               Germany   
4228            vinod_gupta                    NaN                 India   
4229         anil_rai_gupta                    NaN                 India   
4230         willi_liebherr                    NaN           Switzerland   
4231        isolde_liebherr                    NaN           Switzerland   

           state headquarters                                        source  \
0          Texas           TX                                  damaged cars   
1     California           CA  television, Univision; television; Univision   
2            NaN          NaN                                       banking   
3            NaN          NaN                           retail, real estate   
4            NaN          NaN                              timber, palm oil   
...          ...          ...                                           ...   
4227         NaN          NaN                            software; Software   
4228         NaN          NaN                          electrical equipment   
4229         NaN          NaN                          electrical equipment   
4230         NaN          NaN                    Construction; construction   
4231         NaN          NaN                    Construction; construction   

                          industry                      image_uri   age  \
0                       Automotive                         no-pic  52.0   
1     Media & Entertainment; Media    a-jerrold-perenchio; no-pic  91.0   
2                          Finance  abdul-aziz-al-ghurair; no-pic  67.0   
3                 Fashion & Retail                         no-pic  52.0   
4                    Manufacturing                   abdul-rasyid  64.0   
...                            ...                            ...   ...   
4227                    Technology                            NaN   NaN   
4228                 Manufacturing                            NaN   NaN   
4229                 Manufacturing                            NaN   NaN   
4230    Construction & Engineering                            NaN   NaN   
4231    Construction & Engineering                            NaN   NaN   

     gender   last_name  
0         M       Adair  
1         M   Perenchio  
2         M  Al Ghurair  
3         M    Alhokair  
4         M       Abdul  
...     ...         ...  
4227    NaN     Tschira  
4228    NaN       Gupta  
4229    NaN       Gupta  
4230    NaN    Liebherr  
4231    NaN    Liebherr  

[4232 rows x 11 columns]

In [106]:
fbent_final[fbent_final.country.str.contains(';')].shape

(52, 11)

In [107]:
fbent_final[fbent_final.country.str.contains(';')]['country'].values

array(['United States; Colombia', 'Israel; Kazakhstan', 'India; Thailand',
       'Germany; United States', 'Czechia; Czech Republic',
       'Lebanon; Saudi Arabia', 'Lebanon; Switzerland', 'Georgia; Russia',
       'Canada; China', 'Hong Kong; China', 'Hong Kong; China',
       'Czechia; Czech Republic', 'Brazil; United States',
       'United States; Russia', 'Switzerland; Germany',
       'Switzerland; Germany', 'Macau; Macao', 'Hong Kong; China',
       'Israel; Russia', 'Switzerland; Australia', 'Singapore; Taiwan',
       'United States; Canada', 'Cyprus; Norway',
       'Czechia; Czech Republic', 'China; Hong Kong', 'Singapore; China',
       'Monaco; Brazil', 'United States; China',
       'Czechia; Czech Republic', 'United States; South Korea',
       'Eswatini (Swaziland); Swaziland', 'China; Hong Kong',
       'Ireland; India', 'Belgium; Kazakhstan', 'Czechia; Czech Republic',
       'Czechia; Czech Republic', 'Czechia; Czech Republic',
       'Czechia; Czech Republic', 'Au

In [108]:
# TODO: double check those reported with in different countries within etl notebook for forbes

In [109]:
hurun_ent = pd.read_csv('../intermediate/hurun/ddf--entities--person.csv')

In [110]:
hurun_map = all_ids[['hurun_id', 'gm_id']].dropna(how='any').set_index('hurun_id')['gm_id'].to_dict()

In [111]:
hurun_ent

person  \
0           abdulla_ahmed_al_ghurair   
1                 abdulla_al_futtaim   
2                  abdullah_al_rajhi   
3     abdullah_bin_sulaiman_al_rajhi   
4                   abdulsamad_rabiu   
...                              ...   
5023                    zuo_xiaoping   
5024                    zuo_zongshen   
5025     zuo_zongshen_and_yuan_dexiu   
5026                       zygi_wilf   
5027              zygmunt_solorz_zak   

                                                   name       birth birth_1  \
0     Abdulla Ahmed Al Ghurair;Abdulla Ahmed Al Ghur...         NaN     NaN   
1                                    Abdulla Al Futtaim         NaN     NaN   
2                                     Abdullah Al Rajhi         NaN     NaN   
3                        Abdullah bin Sulaiman Al Rajhi  1929-01-01  1929.0   
4                                      Abdulsamad Rabiu  1960-08-04  1960.0   
...                                                 ...         ...     ...   
5023                                       Zuo Xiaoping  1973-01-01  1973.0   
5024                 Zuo Zongshen;Zuo Zongshen & family  1953-01-01  1953.0   
5025                          Zuo Zongshen & Yuan Dexiu         NaN     NaN   
5026                                          Zygi Wilf  1950-04-22  1950.0   
5027                                 Zygmunt Solorz-Zak  1956-08-04  1956.0   

        birth_2 birth_3   age     sex           permanent_country  \
0     1955;1956     NaN   NaN    Male  United Arab Emirates-Dubai   
1           NaN     NaN   NaN    Male  United Arab Emirates-Dubai   
2           NaN     NaN   NaN    Male         Saudi Arabia-Riyadh   
3           NaN     NaN  93.0    Male         Saudi Arabia-Riyadh   
4     1960;1950     NaN  62.0    Male               Nigeria-Lagos   
...         ...     ...   ...     ...                         ...   
5023        NaN     NaN  49.0  Female      China-Guangdong-Foshan   
5024       1953     NaN  69.0    Male             China-Chongqing   
5025       1953     NaN   NaN     NaN                         NaN   
5026       1951     NaN  72.0    Male      United States-Millburn   
5027       1957     NaN  66.0    Male               Poland-Warsaw   

                                               industry  \
0                                               Banking   
1     Automobile Distribution;Automobiles & Componen...   
2                                               Banking   
3                                               Banking   
4     Construction Materials;Conglomerate;Multi Bran...   
...                                                 ...   
5023  Construction Materials;Industrial Products;Pla...   
5024  Auto Components;Motorcycles, Real Estate;Motor...   
5025                           Motorcycles, Real estate   
5026                                 Sports;Real Estate   
5027                                Media;Entertainment   

                                                company               country  \
0                                          Mashreq Bank  United Arab Emirates   
1                                            Al Futtaim  United Arab Emirates   
2                       Al Rajhi Banking And Investment          Saudi Arabia   
3     Al Rajhi Banking & Investment;Al Rajhi Banking...          Saudi Arabia   
4                                               BUA;Bua               Nigeria   
...                                                 ...                   ...   
5023                                          Lesso;L&S                 China   
5024                                           Zongshen                 China   
5025                                           Zongshen                   NaN   
5026                     Minnesota Vikings;Garden Homes         United States   
5027                                     Cyfrowy Polsat                Poland   

      geo  birth_comb  
0     are      1955.0  
1     are         NaN  
2     sau        

In [112]:
hurun_ent[hurun_ent['person'] == 'adam_neuman']

person         name birth birth_1 birth_2 birth_3  age  sex  \
16  adam_neuman  Adam Neuman   NaN     NaN    1980     NaN  NaN  NaN   

   permanent_country                industry company country  geo  birth_comb  
16               NaN  Real Estate;Technology  WeWork     NaN  NaN      1980.0

In [113]:
def _f(x):
    if x in hurun_map.keys():
        return hurun_map[x]
    else:
        print(x)
        return np.nan

hurun_ent['gm_id'] = hurun_ent['person'].map(_f)

In [114]:
# map country name to geo. 
geo_syns = pd.read_csv('../../../ddf--open_numbers/ddf--synonyms--geo.csv')
geo_map = geo_syns.set_index('synonym')['geo'].to_dict()

geo_map['Eswatini (Swaziland)'] = 'swz'

In [115]:
def _f(x):
    if pd.isnull(x):
        return x
    else:
        if ';' in x:
            print(x)
            x_ = x.split(';')[0].strip()  ## first one is latest
            return geo_map[x_]
        else:
            return geo_map[x]

fbent_final['geo'] = fbent_final['country'].map(_f)

United States; Colombia
Israel; Kazakhstan
India; Thailand
Germany; United States
Czechia; Czech Republic
Lebanon; Saudi Arabia
Lebanon; Switzerland
Georgia; Russia
Canada; China
Hong Kong; China
Hong Kong; China
Czechia; Czech Republic
Brazil; United States
United States; Russia
Switzerland; Germany
Switzerland; Germany
Macau; Macao
Hong Kong; China
Israel; Russia
Switzerland; Australia
Singapore; Taiwan
United States; Canada
Cyprus; Norway
Czechia; Czech Republic
China; Hong Kong
Singapore; China
Monaco; Brazil
United States; China
Czechia; Czech Republic
United States; South Korea
Eswatini (Swaziland); Swaziland
China; Hong Kong
Ireland; India
Belgium; Kazakhstan
Czechia; Czech Republic
Czechia; Czech Republic
Czechia; Czech Republic
Czechia; Czech Republic
Austria; France
United Kingdom; Canada
Lebanon; Saudi Arabia
Hong Kong; China
Monaco; Italy
Cyprus; Greece
Cyprus; India
Canada; United Kingdom
Ukraine; Russia
United States; Russia
Germany; Russia
Hong Kong; China
Israel; Russia

In [116]:
fbent_final

person                   name               country  \
0            a_jayson_adair        A. Jayson Adair         United States   
1       a_jerrold_perenchio   A. Jerrold Perenchio         United States   
2     abdul_aziz_al_ghurair  Abdul Aziz Al Ghurair  United Arab Emirates   
3     abdul_majeed_alhokair  Abdul Majeed Alhokair          Saudi Arabia   
4              abdul_rasyid           Abdul Rasyid             Indonesia   
...                     ...                    ...                   ...   
4227         harald_tschira                    NaN               Germany   
4228            vinod_gupta                    NaN                 India   
4229         anil_rai_gupta                    NaN                 India   
4230         willi_liebherr                    NaN           Switzerland   
4231        isolde_liebherr                    NaN           Switzerland   

           state headquarters                                        source  \
0          Texas           TX                                  damaged cars   
1     California           CA  television, Univision; television; Univision   
2            NaN          NaN                                       banking   
3            NaN          NaN                           retail, real estate   
4            NaN          NaN                              timber, palm oil   
...          ...          ...                                           ...   
4227         NaN          NaN                            software; Software   
4228         NaN          NaN                          electrical equipment   
4229         NaN          NaN                          electrical equipment   
4230         NaN          NaN                    Construction; construction   
4231         NaN          NaN                    Construction; construction   

                          industry                      image_uri   age  \
0                       Automotive                         no-pic  52.0   
1     Media & Entertainment; Media    a-jerrold-perenchio; no-pic  91.0   
2                          Finance  abdul-aziz-al-ghurair; no-pic  67.0   
3                 Fashion & Retail                         no-pic  52.0   
4                    Manufacturing                   abdul-rasyid  64.0   
...                            ...                            ...   ...   
4227                    Technology                            NaN   NaN   
4228                 Manufacturing                            NaN   NaN   
4229                 Manufacturing                            NaN   NaN   
4230    Construction & Engineering                            NaN   NaN   
4231    Construction & Engineering                            NaN   NaN   

     gender   last_name  geo  
0         M       Adair  usa  
1         M   Perenchio  usa  
2         M  Al Ghurair  are  
3         M    Alhokair  sau  
4         M       Abdul  idn  
...     ...         ...  ...  
4227    NaN     Tschira  deu  
4228    NaN       Gupta  ind  
4229    NaN       Gupta  ind  
4230    NaN    Liebherr  che  
4231    NaN    Liebherr  che  

[4232 rows x 12 columns]

In [117]:
forbes_ent_1 = fbent_final.copy()
forbes_ent_1 = forbes_ent_1.dropna(subset=['person'])

In [118]:
hurun_ent_1 = hurun_ent.copy()
hurun_ent_1 = hurun_ent_1.dropna(subset=['gm_id'])

In [119]:
hurun_ent_1 = hurun_ent_1.drop(columns=['person']).set_index('gm_id')
hurun_ent_1.index.name = 'person'
hurun_ent_1 = hurun_ent_1.reset_index()

In [120]:
comb_ent = pd.concat([forbes_ent_1, hurun_ent_1], ignore_index=True)

In [121]:
comb_ent

person                                name  \
0                  a_jayson_adair                     A. Jayson Adair   
1             a_jerrold_perenchio                A. Jerrold Perenchio   
2           abdul_aziz_al_ghurair               Abdul Aziz Al Ghurair   
3           abdul_majeed_alhokair               Abdul Majeed Alhokair   
4                    abdul_rasyid                        Abdul Rasyid   
...                           ...                                 ...   
9255                 zuo_xiaoping                        Zuo Xiaoping   
9256                 zuo_zongshen  Zuo Zongshen;Zuo Zongshen & family   
9257  zuo_zongshen_and_yuan_dexiu           Zuo Zongshen & Yuan Dexiu   
9258                    zygi_wilf                           Zygi Wilf   
9259           zygmunt_solorz_zak                  Zygmunt Solorz-Zak   

                   country       state headquarters  \
0            United States       Texas           TX   
1            United States  California           CA   
2     United Arab Emirates         NaN          NaN   
3             Saudi Arabia         NaN          NaN   
4                Indonesia         NaN          NaN   
...                    ...         ...          ...   
9255                 China         NaN          NaN   
9256                 China         NaN          NaN   
9257                   NaN         NaN          NaN   
9258         United States         NaN          NaN   
9259                Poland         NaN          NaN   

                                            source  \
0                                     damaged cars   
1     television, Univision; television; Univision   
2                                          banking   
3                              retail, real estate   
4                                 timber, palm oil   
...                                            ...   
9255                                           NaN   
9256                                           NaN   
9257                                           NaN   
9258                                           NaN   
9259                                           NaN   

                                               industry  \
0                                            Automotive   
1                          Media & Entertainment; Media   
2                                               Finance   
3                                      Fashion & Retail   
4                                         Manufacturing   
...                                                 ...   
9255  Construction Materials;Industrial Products;Pla...   
9256  Auto Components;Motorcycles, Real Estate;Motor...   
9257                           Motorcycles, Real estate   
9258                                 Sports;Real Estate   
9259                                Media;Entertainment   

                          image_uri   age gender   last_name  geo       birth  \
0                            no-pic  52.0      M       Adair  usa         NaN   
1       a-jerrold-perenchio; no-pic  91.0      M   Perenchio  usa         NaN   
2     abdul-aziz-al-ghurair; no-pic  67.0      M  Al Ghurair  are         NaN   
3                            no-pic  52.0      M    Alhokair  sau         NaN   
4                      abdul-rasyid  64.0      M       Abdul  idn         NaN   
...                             ...   ...    ...         ...  ...         ...   
9255                            NaN  49.0    NaN         NaN  chn  1973-01-01   
9256                            NaN  69.0    NaN         NaN  chn  1953-01-01   
9257                            NaN   NaN    NaN         NaN  NaN         NaN   
9258                            NaN  72.0    NaN         NaN  usa  1950-04-22   
9259                            NaN  66.0    NaN         NaN  pol  1956-08-04   

     birth_1 birth_2 birth_3     sex       permanent_country  \
0        NaN     NaN     NaN     NaN                     NaN   
1        NaN     NaN     NaN  

In [122]:
gs = comb_ent.groupby('person')

In [123]:
gs.get_group('jeff_bezos')

person        name        country       state headquarters  \
1581  jeff_bezos  Jeff Bezos  United States  Washington           WA   
5990  jeff_bezos  Jeff Bezos  United States         NaN          NaN   

                  source                  industry   image_uri   age gender  \
1581  Amazon; Amazon.com                Technology  jeff-bezos  58.0      M   
5990                 NaN  E-Commerce;Online Retail         NaN  58.0    NaN   

     last_name  geo       birth birth_1 birth_2 birth_3   sex  \
1581     Bezos  usa         NaN     NaN     NaN     NaN   NaN   
5990       NaN  usa  1964-01-12  1964.0    1964    1964  Male   

                           permanent_country company  birth_comb  
1581                                     NaN     NaN         NaN  
5990  United States-Washington State-Seattle  Amazon      1964.0

In [124]:
gs.get_group('cai_kui')

person                                       name    country    state  \
430   cai_kui                                    Cai Kui      China  Sichuan   
4686  cai_kui  Cai Kui;Cai Kui & Family;Cai Kui & family  Hong Kong      NaN   

     headquarters       source               industry image_uri   age gender  \
430           NaN  real estate            Real Estate   cai-kui  59.0      M   
4686          NaN          NaN  Developer;Real Estate       NaN  61.0    NaN   

     last_name  geo       birth birth_1 birth_2 birth_3   sex  \
430        Cai  chn         NaN     NaN     NaN     NaN   NaN   
4686       NaN  hkg  1961-01-01  1961.0     NaN     NaN  Male   

     permanent_country  company  birth_comb  
430                NaN      NaN         NaN  
4686   China-Hong Kong  Longfor      1961.0

In [125]:
hurun_ent[hurun_ent['gm_id'] == 'cai_kui']

person                                       name       birth birth_1  \
454  cai_kui  Cai Kui;Cai Kui & Family;Cai Kui & family  1961-01-01  1961.0   

    birth_2 birth_3   age   sex permanent_country               industry  \
454     NaN     NaN  61.0  Male   China-Hong Kong  Developer;Real Estate   

     company    country  geo  birth_comb    gm_id  
454  Longfor  Hong Kong  hkg      1961.0  cai_kui

In [126]:
# NOTE: because forbes is in front of hurun list,
# the combined result will have forbes propertties in front.

def group_func(df):
    res = dict()
    for c in df.columns:
        if c not in ['wealth', 'ranking', 'year']:
            values = [str(x).strip() for x in df[c].dropna().values]
            if values:
                if c == 'geo' and ('hkg' in values or 'twn' in values or 'mac' in values):
                    values.insert(0, 'chn')
                values = pd.Series(values).drop_duplicates(keep='first').values
                res[c] = ";".join(values)
            else:
                res[c] = ''
    return pd.Series(res)

In [127]:
group_func(gs.get_group('cai_kui'))

person                                                         cai_kui
name                 Cai Kui;Cai Kui;Cai Kui & Family;Cai Kui & family
country                                                China;Hong Kong
state                                                          Sichuan
headquarters                                                          
source                                                     real estate
industry                             Real Estate;Developer;Real Estate
image_uri                                                      cai-kui
age                                                          59.0;61.0
gender                                                               M
last_name                                                          Cai
geo                                                            chn;hkg
birth                                                       1961-01-01
birth_1                                                         1961.0
birth_

In [128]:
# TODO: for now keep all columns.. need to cleanup

In [129]:
res_ent = gs.apply(group_func)

In [130]:
res_ent

person  \
person                                                     
a_jayson_adair                            a_jayson_adair   
a_jerrold_perenchio                  a_jerrold_perenchio   
abdul_aziz_al_ghurair              abdul_aziz_al_ghurair   
abdul_majeed_alhokair              abdul_majeed_alhokair   
abdul_rasyid                                abdul_rasyid   
...                                                  ...   
zuo_zongshen                                zuo_zongshen   
zuo_zongshen_and_yuan_dexiu  zuo_zongshen_and_yuan_dexiu   
zuowen_song                                  zuowen_song   
zygi_wilf                                      zygi_wilf   
zygmunt_solorz_zak                    zygmunt_solorz_zak   

                                                                          name  \
person                                                                           
a_jayson_adair                                                 A. Jayson Adair   
a_jerrold_perenchio                     A. Jerrold Perenchio;Jerrold Perenchio   
abdul_aziz_al_ghurair                                    Abdul Aziz Al Ghurair   
abdul_majeed_alhokair                                    Abdul Majeed Alhokair   
abdul_rasyid                                                      Abdul Rasyid   
...                                                                        ...   
zuo_zongshen                                Zuo Zongshen;Zuo Zongshen & family   
zuo_zongshen_and_yuan_dexiu                          Zuo Zongshen & Yuan Dexiu   
zuowen_song                  Zuowen Song;Song Zuowen;Song Zuowen & Family;S...   
zygi_wilf                                                            Zygi Wilf   
zygmunt_solorz_zak                                          Zygmunt Solorz-Zak   

                                          country       state headquarters  \
person                                                                       
a_jayson_adair                      United States       Texas           TX   
a_jerrold_perenchio                 United States  California           CA   
abdul_aziz_al_ghurair        United Arab Emirates                            
abdul_majeed_alhokair                Saudi Arabia                            
abdul_rasyid                            Indonesia                            
...                                           ...         ...          ...   
zuo_zongshen                                China                            
zuo_zongshen_and_yuan_dexiu                                                  
zuowen_song                                 China                            
zygi_wilf                           United States                            
zygmunt_solorz_zak                         Poland                            

                                                                   source  \
person                                                                      
a_jayson_adair                                               damaged cars   
a_jerrold_perenchio          television, Univision; television; Univision   
abdul_aziz_al_ghurair                                             banking   
abdul_majeed_alhokair                                 retail, real estate   
abdul_rasyid                                             timber, palm oil   
...                                                                   ...   
zuo_zongshen                                                                
zuo_zongshen_and_yuan_dexiu                                                 
zuowen_song                                         aluminum, diversified   
zygi_wilf                                                                   
zygmunt_solorz_zak                            TV broadcasting; TV station   

                                                                      industry  \
person                                                                           
a_jayson_adair       

In [131]:
# res_ent = res_ent.drop(columns=['gm_id'])

In [132]:
res_ent.loc['cai_kui']

person                                                         cai_kui
name                 Cai Kui;Cai Kui;Cai Kui & Family;Cai Kui & family
country                                                China;Hong Kong
state                                                          Sichuan
headquarters                                                          
source                                                     real estate
industry                             Real Estate;Developer;Real Estate
image_uri                                                      cai-kui
age                                                          59.0;61.0
gender                                                               M
last_name                                                          Cai
geo                                                            chn;hkg
birth                                                       1961-01-01
birth_1                                                         1961.0
birth_

In [133]:
res_ent = res_ent.drop(columns=['person'])

In [134]:
# res_ent.index.name = 'person'

In [135]:
res_ent['countries'] = res_ent['geo']
res_ent['names'] = res_ent['name']

In [136]:
def _f(x):
    if isinstance(x, str):
        if ';' in x:
            return x.split(';')[0].strip()
    return x

res_ent['name'] = res_ent['names'].map(_f)

In [137]:
res_ent = res_ent.drop(columns=['geo'])

In [138]:
res_ent = res_ent.drop(columns=['country'])

In [139]:
res_ent.columns

Index(['name', 'state', 'headquarters', 'source', 'industry', 'image_uri',
       'age', 'gender', 'last_name', 'birth', 'birth_1', 'birth_2', 'birth_3',
       'sex', 'permanent_country', 'company', 'birth_comb', 'countries',
       'names'],
      dtype='object')

In [140]:
# drop those entities without country
res_ent = res_ent[~pd.isnull(res_ent['countries'])]

In [141]:
res_ent

name       state  \
person                                                               
a_jayson_adair                         A. Jayson Adair       Texas   
a_jerrold_perenchio               A. Jerrold Perenchio  California   
abdul_aziz_al_ghurair            Abdul Aziz Al Ghurair               
abdul_majeed_alhokair            Abdul Majeed Alhokair               
abdul_rasyid                              Abdul Rasyid               
...                                                ...         ...   
zuo_zongshen                              Zuo Zongshen               
zuo_zongshen_and_yuan_dexiu  Zuo Zongshen & Yuan Dexiu               
zuowen_song                                Zuowen Song               
zygi_wilf                                    Zygi Wilf               
zygmunt_solorz_zak                  Zygmunt Solorz-Zak               

                            headquarters  \
person                                     
a_jayson_adair                        TX   
a_jerrold_perenchio                   CA   
abdul_aziz_al_ghurair                      
abdul_majeed_alhokair                      
abdul_rasyid                               
...                                  ...   
zuo_zongshen                               
zuo_zongshen_and_yuan_dexiu                
zuowen_song                                
zygi_wilf                                  
zygmunt_solorz_zak                         

                                                                   source  \
person                                                                      
a_jayson_adair                                               damaged cars   
a_jerrold_perenchio          television, Univision; television; Univision   
abdul_aziz_al_ghurair                                             banking   
abdul_majeed_alhokair                                 retail, real estate   
abdul_rasyid                                             timber, palm oil   
...                                                                   ...   
zuo_zongshen                                                                
zuo_zongshen_and_yuan_dexiu                                                 
zuowen_song                                         aluminum, diversified   
zygi_wilf                                                                   
zygmunt_solorz_zak                            TV broadcasting; TV station   

                                                                      industry  \
person                                                                           
a_jayson_adair                                                      Automotive   
a_jerrold_perenchio                 Media & Entertainment; Media;Entertainment   
abdul_aziz_al_ghurair                                                  Finance   
abdul_majeed_alhokair                                         Fashion & Retail   
abdul_rasyid                                                     Manufacturing   
...                                                                        ...   
zuo_zongshen                 Auto Components;Motorcycles, Real Estate;Motor...   
zuo_zongshen_and_yuan_dexiu                           Motorcycles, Real estate   
zuowen_song                  Diversified;Non-ferrous metals processing;Meta...   
zygi_wilf                                                   Sports;Real Estate   
zygmunt_solorz_zak            Media & Entertainment; Media;Media;Entertainment   

                                                 image_uri        age gender  \
person                                                                         
a_jayson_adair                                      no-pic       52.0      M   
a_jerrold_perenchio            a-jerrold-perenchio; no-pic       91.0      M   
abdul_aziz_al_ghurair        abdul-aziz-al-ghurair; no-pic       67.0      M   
abdul_majeed_alhokair                               no-pic       52.0      M   
abdul_rasyid     

In [142]:
res_ent.to_csv('../../ddf--entities--person.csv')

In [143]:
res_ent

name       state  \
person                                                               
a_jayson_adair                         A. Jayson Adair       Texas   
a_jerrold_perenchio               A. Jerrold Perenchio  California   
abdul_aziz_al_ghurair            Abdul Aziz Al Ghurair               
abdul_majeed_alhokair            Abdul Majeed Alhokair               
abdul_rasyid                              Abdul Rasyid               
...                                                ...         ...   
zuo_zongshen                              Zuo Zongshen               
zuo_zongshen_and_yuan_dexiu  Zuo Zongshen & Yuan Dexiu               
zuowen_song                                Zuowen Song               
zygi_wilf                                    Zygi Wilf               
zygmunt_solorz_zak                  Zygmunt Solorz-Zak               

                            headquarters  \
person                                     
a_jayson_adair                        TX   
a_jerrold_perenchio                   CA   
abdul_aziz_al_ghurair                      
abdul_majeed_alhokair                      
abdul_rasyid                               
...                                  ...   
zuo_zongshen                               
zuo_zongshen_and_yuan_dexiu                
zuowen_song                                
zygi_wilf                                  
zygmunt_solorz_zak                         

                                                                   source  \
person                                                                      
a_jayson_adair                                               damaged cars   
a_jerrold_perenchio          television, Univision; television; Univision   
abdul_aziz_al_ghurair                                             banking   
abdul_majeed_alhokair                                 retail, real estate   
abdul_rasyid                                             timber, palm oil   
...                                                                   ...   
zuo_zongshen                                                                
zuo_zongshen_and_yuan_dexiu                                                 
zuowen_song                                         aluminum, diversified   
zygi_wilf                                                                   
zygmunt_solorz_zak                            TV broadcasting; TV station   

                                                                      industry  \
person                                                                           
a_jayson_adair                                                      Automotive   
a_jerrold_perenchio                 Media & Entertainment; Media;Entertainment   
abdul_aziz_al_ghurair                                                  Finance   
abdul_majeed_alhokair                                         Fashion & Retail   
abdul_rasyid                                                     Manufacturing   
...                                                                        ...   
zuo_zongshen                 Auto Components;Motorcycles, Real Estate;Motor...   
zuo_zongshen_and_yuan_dexiu                           Motorcycles, Real estate   
zuowen_song                  Diversified;Non-ferrous metals processing;Meta...   
zygi_wilf                                                   Sports;Real Estate   
zygmunt_solorz_zak            Media & Entertainment; Media;Media;Entertainment   

                                                 image_uri        age gender  \
person                                                                         
a_jayson_adair                                      no-pic       52.0      M   
a_jerrold_perenchio            a-jerrold-perenchio; no-pic       91.0      M   
abdul_aziz_al_ghurair        abdul-aziz-al-ghurair; no-pic       67.0      M   
abdul_majeed_alhokair                               no-pic       52.0      M   
abdul_rasyid     

In [144]:
res_ent.countries.unique()

array(['usa', 'are', 'sau', 'idn', 'nga', 'ind', 'bra', 'chn;hkg', '',
       'isr;usa', 'rou', 'dnk', 'tur', 'chn', 'gbr', 'rus', 'tha', 'jpn',
       'can', 'fra', 'gbr;che', 'che', 'aus', 'deu', 'bel', 'esp', 'mex',
       'per', 'ita', 'arg', 'isr', 'kaz', 'nor', 'prt', 'rus;gbr', 'swe',
       'nld', 'rus;che', 'zaf', 'ind;tha', 'chl', 'mys', 'mar', 'bra;gbr',
       'chn;twn', 'nor;usa', 'deu;usa', 'cze', 'ury', 'phl', 'ukr',
       'ind;gbr', 'fra;che', 'fin', 'can;usa', 'mco', 'grc;che', 'sgp',
       'deu;che', 'ita;che', 'lbn;sau', 'lbn;che', 'kor', 'kwt;usa',
       'col', 'che;fra', 'swe;che', 'geo', 'npl', 'usa;aus', 'usa;kor',
       'ind;are', 'aus;bhs', 'vnm', 'gbr;usa', 'fra;gbr', 'can;chn',
       'nld;gbr', 'mys;khm', 'chn;twn;usa', 'deu;gbr', 'lie', 'nzl;are',
       'cyp;mco', 'rou;usa', 'esp;che', 'pol', 'chn;mac', 'isl;dnk',
       'col;bra', 'usa;chn', 'irl', 'aut', 'rus;sgp', 'rus;mco',
       'idn;usa', 'bra;sgp', 'bel;mco', 'isr;mco', 'lbn;fra', 'qat',
      

In [145]:
res_ent[res_ent.countries.str.contains(';')]['countries'].to_dict()

{'adam_kwok': 'chn;hkg',
 'adam_kwok_kai_fai': 'chn;hkg',
 'adam_neumann': 'isr;usa',
 'alan_howard': 'gbr;che',
 'albert_yeung': 'chn;hkg',
 'alexei_kuzmichev': 'rus;gbr',
 'alisher_usmanov': 'rus;che',
 'allan_wong': 'chn;hkg',
 'aloke_lohia': 'ind;tha',
 'andre_esteves': 'bra;gbr',
 'andre_koo_sr': 'chn;twn',
 'andreas_halvorsen': 'nor;usa',
 'andreas_von_bechtolsheim': 'deu;usa',
 'andrew_currie': 'gbr;che',
 'angela_leong': 'chn;hkg',
 'anil_agarwal': 'ind;gbr',
 'anne_beaufour': 'fra;che',
 'antonio_luiz_seabra': 'bra;gbr',
 'apoorva_mehta': 'can;usa',
 'archie_hwang': 'chn;twn',
 'aristotelis_mistakidis': 'grc;che',
 'august_von_finck': 'deu;che',
 'augusto_giorgio_perfetti': 'ita;che',
 'ayman_hariri': 'lbn;sau',
 'bahaa_hariri': 'lbn;che',
 'barry_lam': 'chn;twn',
 'bassam_alghanim': 'kwt;usa',
 'benjamin_de_rothschild': 'che;fra',
 'bertil_hult': 'swe;che',
 'billy_kan': 'chn;hkg',
 'blair_parry_okeden': 'usa;aus',
 'bom_kim': 'usa;kor',
 'br_shetty': 'ind;are',
 'brett_blund